# Introduction

Word2Vec provides to obtain not just some vectors of words but vectors saving the semantics of the words. A wonderful property of saved semantics of the words is in obtained embeddings space the words with similar sence are close to each other. Moreover, the words which appear in similar <b>contexts</b> (such as tea, coffee, water, etc.) are also close to each other in the same space (but in some different way). Different words are quite far from each other in the embeddings space. So, as a metric of closeness of words to each other or of a metric of a common context we can use distance in embedding space in this approach.

Word2Vec has 2 method of implementation:
* <b>CBOW</b> (Continuous Bag-Of-Words). By a context (words around) of the word we are trying to predict the central word.
* <b>Skip-gram</b>. By the central word we are trying to predict words from a context.

<center><img src="CBOW.png"></center>

Practice demonstrates that <b>Skip-gram</b> architecture works better than <b>CBOW</b>.

# Data downloading and data preprocessing

## Downloading

This text is a file of a purified file text from Wikipedia from Matt Mahoni

In [ ]:
#!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip

## Preprocessing

In [ ]:
with open("./text8/text8") as f:
    text = f.read()
    
print(text[:100])

### Purifying of data

Firstly, we need to purify our text. We need to:
* Transfrom any punctuation marks to tokens, that's why we exchange dots to \<PERIOD\>.
* Delete any words which have ever met in the text five of less times. It must strongly decrease noise in our data and must strongly increase quality of the model.
* (optionally) Have possibility of returning of list of the words.

In [ ]:
import re
from collections import Counter

def preprocess(text):

    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    text = text.replace(':', ' <COLON> ')
    words = text.split()

    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]

    return trimmed_words


def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: Two dictionaries, vocab_to_int, int_to_vocab
    """
    word_counts = Counter(words)
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    # create int_to_vocab dictionaries
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

In [ ]:
words = preprocess(text)
print(f"Belonging to the text: {words[:30]}")
print(f"Total words in the text: {len(words)}")
print(f"Unique words: {len(set(words))}")

### Mapping of words to indices

Here we create two dictionaries for transforming words to integer numbers and reverse. Integer numbers are assigned in order to decreasing of frequency, that's why to the most frequent word (the) is assigned number 0, the next word has number -1 and so on.

Then when we have dictionaries, words are transformed into numbers and saved into `int_words` list.

In [ ]:
vocab_to_int, int_to_vocab = create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

print(int_words[:30])

### Subsampling

Frequently encountered words such as "the", "of", "for" not provide a significant context to closely placed words. If we throw out some of them, we will be able to delete a piece of noise from our data and obtain faster training of the model and better representation. This process is sometimes called <b>subsmapling</b>. For each word $\omega_i$ in training sample we throw out it with probability equals

$$P(\omega_i) = 1 - \sqrt{\frac{t}{f(\omega_i)}}$$

where $t$ - a threshold parameter, $f(\omega_i)$ - frequency of the word $\omega_i$ in the whole set of the data.

In [ ]:
import random
from collections import Counter

import numpy as np
from tqdm.auto import tqdm, trange

In [ ]:
threshold = 1e-5
word_counts = Counter(int_words) # dictionary with number of appearences for each word
print(f"42-th word appears in the text {word_counts[42]} times")

train_words = []

for int_w in tqdm(int_words):
    if random.random() < (threshold / (word_counts[int_w] / len(int_words))) ** 0.5:
        train_words.append(int_w)
    else:
        continue

In [ ]:
print(int_words[:15])
print(train_words[:15])

### Making of training couples

Now, after preliminary data preprocessing we need to make training couples to train the model. In skip-gram architecture for each word in the text we want to define envioroning context and take all the words in the window of size $C$ around this word.

But it's reasonable to hold $C$ unfixed. It helps hold the model undependent from size of the window. If we, for instance, take $C = 5$, then for each train word we choose randomly some number $R$ in an interval $[1: C]$ and thn use $R$ as size of the window (use $R$ next and $R$ previous words as right labels).

In [ ]:
from typing import List, Tuple

def get_target(words: List[int], idx: int, window_size: int=5) -> List[int]:
    """
    words: a text represented as a sequence of words indices
    idx: index of the central word that is used to make a batch
    window_size: controls the size of the window for each word
    
    returns list of words in a window of a window_size size
    """
    
    r = random.randint(1, window_size)
    target = words[max(0, idx - r): idx] + words[idx + 1: min(len(words), idx + r)]
    return target

In [ ]:
int_text = [i for i in range(10)]
idx = 5
target = get_target(int_text, idx=idx, window_size=5)
print("Input: ", int_text)
print(f"Index of interest: {idx}")
print("Target: ", target)

### Generating batches for training

In [ ]:
def get_batches(words: List[int], batch_size: int, window_size=5) -> Tuple[List[int], List[int]]:
    
    for i in range(0, len(words) // batch_size * batch_size, batch_size):
        x, y = [], []
        batch = words[i: i + batch_size]
        for j in range(len(batch)):
            batch_x = batch[j]
            batch_y = get_target(batch, j, window_size)
            y.extend(batch_y)
            x.extend([batch_x] * len(batch_y))
        yield x, y

In [ ]:
int_text = [i for i in range(20)]

batch_gen = get_batches(int_text, batch_size=4, window_size=5)
x, y = next(batch_gen)

print(f"x: {x}")
print(f"y: {y}")

# Skip-gram training

Below you can see a chematic representation of our network:

<center><img src="skip-gram.png"></center>

* Input words are transmitted as batches of indices of input words. Target variables for each input word - word number from the context.
* Input words is processed by linear layer `vocab_size` x `embed_size`.
* Obtained embeddings is processed by output linear layer of size `embed_size` x `vocab_size` and loss for classification problems is applyed to outputs.

The main idea of the method is training of a weights matrix of embeddings layer and searching for effective representations of our layers. After training we can throw out softmax layer because we don't need to make predictions based on this network we just need embeddings matrix to use it in another networks which we build with using of this dataset.

<b>A couple of words about learning control - validation</b>.

We need to choose a few unusual words. Then we can print the neares to them using cosine similarity:
$$similarity = cos(\theta) = \frac{\overrightarrow{a} \cdot \overrightarrow{b}}{|\overrightarrow{a}||\overrightarrow{b}|}$$

We can encode the words of validation dataset as vectors $\overrightarrow{a}$ by using of embeddings table and then calculate similarity to each word vector $\overrightarrow{b}$ in embeddings table. Having similarity we can print the validation words and the words our embeddings matrix semantically similar to these words. This is a good way to check whether our embeddings table join the words.

In [ ]:
def cosine_similarity(embedding, valid_size=16, valid_window=100, device='cpu'):
    embed_vectors = embedding.weight
    
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    valid_examples = np.array(random.sample(range(valid_window), valid_size // 2))
    valid_examples = np.append(valid_examples, 
                              random.sample(range(1000, 1000+valid_window), valid_size // 2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t()) / magnitudes
    
    return valid_examples, similarities

## Training process

In [ ]:
import torch
import torch.optim as optim
from torch import nn

In [ ]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size: int, embed_size: int):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.linear = nn.Linear(embed_size, vocab_size)
    
    def forward(self, x: torch.Tensor):
        out = self.linear(self.embed(x))
        return out

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print_every = 200
steps = 0
n_epochs = 5
batch_size = 1024
embedding_dim = 128

model = SkipGram(len(vocab_to_int), embedding_dim)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

for e in trange(n_epochs, leave=True, desc="Epoch number"):
    pbar = tqdm(
        get_batches(train_words, batch_size),
        leave=False,
        desc="Batch number",
        total=len(train_words) // batch_size
    )
    
    for inputs, targets in pbar:
        steps+=1
        inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
        inputs, targets = inputs.to(device), targets.to(device)
        
        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if steps % print_every == 0:
            valid_examples, valid_similarities = cosine_similarity(model.embed, device=device)
            _, closest_idxs = valid_similarities.topk(6)
            
            valid_examples, closest_idxs = valid_examples.to("cpu"), closest_idxs.to("cpu")
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ", ".join(closest_words))
            print("...")
                

# Analyzing of trained model

We use T-SNE to visualize how our multidimensional words vercors are grouped together.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
embeddings = model.embed.weight.to("cpu").data.numpy()

In [ ]:
viz_words = 600
tsne= TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color="steelblue")
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)